# Simple HMM in NumPy or PyTorch

In [1]:
%run /home/hanzl/f-w/prak/acmodel/matrix.py

In [2]:
A = m([[0.7,0.3,0,0],
       [0,0.9,0.1,0],
       [0,0,0.8,0.2],
       [0,0,0,1]])

In [3]:
# where we can be
x = m([[1,0,0,0]])

In [4]:
# recording length in frames
tmax = 14000

In [5]:
len_x = x.size()[1]

# allocate space for mantissa-like (kept in range) and row-exponent values
alpha_m = m.rowlist((tmax,len_x))
alpha_exp = m.rowlist((tmax,1))

exponent = 0
base = 1000 # base for separation to exp and m (somewhere around 5e-324 starts underflow)

In [6]:
# FAKE b VALUES:
b = m.zeros((tmax,len_x)) + 0.1

In [7]:
x_m = m([[1,0,0,0]]) 

In [8]:
%%time
x_m = m([[1,0,0,0]]) # mantissa-like moderated value (kept in range)
for row in range(tmax):
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1
    alpha_exp[row] = exponent
    alpha_m[row] = x_m
    x_m = x_m@A*b[row]   # FINETUNE WHICH row's b IS USED (ALSO FOR beta)

CPU times: user 1.59 s, sys: 18.4 ms, total: 1.61 s
Wall time: 214 ms


Wall time: 6.77 s for GPU, 1.44 s for CPU (both PyTorch sparse)

GPU: 7.01 torch sparse, 1.12 torch dense

CPU: 1.54 torch sparse, 0.214 torch dense



In [9]:
# Now we will go backward
At = A.T()

In [10]:
beta_m = m.rowlist((tmax,len_x))
beta_exp = m.rowlist((tmax,1))

exponent = 0

In [11]:
x_m = m([[0,0,0,1]])
for row in range(tmax-1,0-1,-1):
    beta_m[row] = x_m
    beta_exp[row] = exponent
    x_m = x_m@At*b[row]
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1

In [12]:
L_m = alpha_m*beta_m # this is .*

In [13]:
L_exp = alpha_exp+beta_exp

In [14]:
L_exp += float(-L_exp.max())

In [15]:
# re-normalize L
for row in range(tmax):
    while float(L_exp[row].val)<0: # renormalize and remember power of base used
        L_m[row] *= 1/base  # invokes setitem which converts L_m to dense
        L_exp[row] += 1

In [16]:
L_m.size()

torch.Size([14000, 4])

In [17]:
L_m # Final result here

m(tensor([[1.0000e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [7.0000e-05, 3.0000e-05, 0.0000e+00, 0.0000e+00],
        [4.9000e-05, 4.8000e-05, 3.0000e-06, 0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e-04],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e-04],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e-04]]))

In [18]:
# test - should be the same now
L_exp.min(), L_exp.max()

(tensor(0.), tensor(0.))